In [1]:
from SSSO.manage import Manager
import time
import numpy as np

In [2]:
manager = Manager(jena_address='http://127.0.0.1:7996/SSSO') # Please modify the triple store address

In [3]:
eqcount = manager.ontology.query("""
SELECT (COUNT(?subject) AS ?triples)
WHERE {
  ?subject rdf:type ?class.
  ?class rdfs:subClassOf ?upperclass.
  ?upperclass rdfs:subClassOf ssso:Equipment.
  
}""")[0][0].toPython()
print('Number of devices: ', eqcount)

Number of devices:  20000


In [4]:
svccount = manager.ontology.query("""
SELECT (COUNT(?subject) AS ?triples)
WHERE {
  ?subject rdf:type ?class.
  ?class rdfs:subClassOf ?upperclass.
  ?upperclass rdfs:subClassOf ssso:Service.
  
}""")[0][0].toPython()
print('Number of services: ', svccount)

Number of services:  180000


In [5]:
concount = manager.ontology.query("""
SELECT (COUNT(?subject) AS ?triples)
WHERE {
  ?subject rdf:type ?class.
  ?class rdfs:subClassOf ?upperclass.
  ?upperclass rdfs:subClassOf ssso:Context.
  
}""")[0][0].toPython()
print('Number of contextual individuals: ', concount)

Number of contextual individuals:  260002


In [6]:
triplecount = manager.ontology.query("""
SELECT (COUNT(?subject) AS ?triples)
WHERE {
  ?subject ?property ?object.
  
}""")[0][0].toPython()
print('Number of triples: ', triplecount)

Number of triples:  1860701


In [7]:
response_time = []
context = {
"Location":"BLD4-F16-M201-2",
"Group":"BLD4-Classified-Room"
}
event_count = 0

In [8]:
# Device/Service Registration
for i in range(10):
    startT = time.time()
    manager.equipment.create(equipmentModel='ae137288-3782-432d-8d83-d5cee733fe4b', context=context)
    endT = time.time()
    response_time.append(endT-startT)
    event_count+=1

In [9]:
# User's Ruquest to enable/disable a service
for i in range(10):
    startT = time.time()
    manager.service.enable('85c72518-c304-463f-b925-d2cb1cf54791', 'user1')
    endT = time.time()
    response_time.append(endT-startT)
    startT = time.time()
    manager.service.disable('85c72518-c304-463f-b925-d2cb1cf54791')
    endT = time.time()
    response_time.append(endT-startT)
    event_count+=2

In [10]:
# Threat Handling
for i in range(10):
    startT = time.time()
    threatUUID = manager.event.report_threat('f25d6fab-a140-42eb-b31b-3b9d94e96b19', 'Unexpected_Occupancy')[1]
    endT = time.time()
    response_time.append(endT-startT)
    startT = time.time()
    manager.event.remove_threat(threatUUID)
    endT = time.time()
    response_time.append(endT-startT)
    event_count+=2

In [11]:
print(f'Average response time for handling {event_count} events in seconds: ', np.mean(response_time))

Average response time for handling 50 events in seconds:  1.7915902709960938


In [12]:
# Save logs
with open('performance-evaluation-results.txt', 'w') as file:
    for item in response_time:
        file.write('%s\n' % item)